<a href="https://colab.research.google.com/github/parthag1201/DocReversalEngine/blob/main/Workflow/WorkflowDiagramV1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
%pip install crewai_tools langchain_community langchain_google_genai fpdf2 markdown2

^C
Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install crewai agentic-ai

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Install the google-generativeai package
%pip install google-generativeai

INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 1.3/1.3 MB 7.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/13.7 MB ? eta -:--:--
   --- ------------------------------------ 1.3/13.7 MB 7.4 MB/s eta 0:00:02
   ------- -------------------------------- 2.6/13.7 MB 6.8 MB/s eta 0:00:02
   ----------- ---------------------------- 3.9/13.7 MB 6.8 MB/s eta 0:00:02
   ---------------- ----------------------- 5.5/13.7 MB 6.8 MB/s eta 0:00:02
   ------------------- -------------------- 6.8/13.7 MB 6.8 MB/s eta 0:00:02
   ------------------------ --------------- 8.4/13.7 MB 6.9 MB/s eta 0:00:01
   ------------------------------ --------- 10.5/13.7 MB 7.2 MB/s eta 0:00:01
   ----------------------------------- ---- 12.1/13.7 MB 7.3 MB/s eta 0:00:01
   --------------

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 2.1.8 requires google-ai-generativelanguage<0.7.0,>=0.6.18, but you have google-ai-generativelanguage 0.6.15 which is incompatible.


In [ ]:
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

c:\Users\akikumar\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# pdf read and write
from fpdf import FPDF
from markdown2 import Markdown

In [ ]:
# Warning Control
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Import crewAI library
from crewai import Agent, Task, Crew, Process
from crewai.project import CrewBase, agent, crew, task
from crewai.agents.agent_builder.base_agent import BaseAgent
from typing import List
from crewai.flow.flow import Flow, listen, start

HTTPSConnectionPool(host='telemetry.crewai.com', port=4319): Max retries exceeded with url: /v1/traces (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1028)')))


In [108]:
import os

# %pip install python-dotenv

from dotenv import load_dotenv
load_dotenv()

apiKey = os.getenv("GOOGLE_API_KEY3")

# print(apiKey)

In [98]:
#setting up LLM

from crewai import LLM

llm = LLM(model="gemini/gemini-2.5-pro",
                             verbose=True,
                             temperature=0.5,
                             api_key = apiKey)

# print(apiKey)


In [42]:
import os
import re
from tkinter import Tk
from tkinter.filedialog import askopenfilename

# Hide the root tkinter window
Tk().withdraw()

# Open file dialog
file_path = askopenfilename(
    title="Select ABAP Code File",
    filetypes=[("Text Files", "*.txt"), ("All Files", "*.*")]
)

abap_code_example = ""
output_html_filename = ""

if file_path:
    with open(file_path, 'r', encoding='utf-8') as f:
        abap_code_example = f.read()
    print("File selected:", file_path)
    print("Code Preview:\n", abap_code_example[:500])  # Show a preview

    # Get sanitized filename without extension
    base_filename = os.path.splitext(os.path.basename(file_path))[0]
    output_html_filename = re.sub(r'[\\/*?:"<>|]', "_", base_filename) + ".html"
else:
    print("No file selected.")
    

In [43]:


# main.py
import os
from crewai import Agent, Task, Crew, Process
from langchain_google_genai import ChatGoogleGenerativeAI
# from crewai_tools import ScrapeWebsiteTool
# --- Agent Definitions ---

# 1. ABAP Code Interpreter Agent
# This agent is the first point of contact with the raw ABAP code.
# Its primary goal is to understand the code's structure, identify key components
# like FORMs, FUNCTIONs, METHODs, and understand the high-level flow.
abap_interpreter = Agent(
    role='ABAP Code Interpreter',
    goal='Analyze the provided ABAP code to understand its structure, main components (like FORMs, METHODs, FUNCTIONs), and the overall program flow.',
    backstory=(
        "As a seasoned ABAP developer with decades of experience, you have an unparalleled ability to read and instantly "
        "comprehend even the most complex and archaic ABAP code. You can see beyond the syntax to the underlying business "
        "logic and program structure. Your task is to distill this complex code into a clear, structured summary that "
        "other agents can use."
    ),
    verbose=True,
    allow_delegation=False,
    llm=llm
)

# 2. Logic Reviewer Agent
# This agent takes the structured summary from the interpreter and focuses on the
# business logic. It identifies conditional statements (IF/ELSE, CASE), loops (DO, WHILE),
# and subroutine calls to map out the decision points and processes.
logic_reviewer = Agent(
    role='Business Logic Analyst',
    goal='Review the structured summary of the ABAP code to identify and map out the core business logic, including conditional paths, loops, and subroutine calls.',
    backstory=(
        "You are a meticulous business analyst who specializes in reverse-engineering legacy systems. You have a keen eye for "
        "detail and can trace the flow of logic through complex code. Your strength is in identifying the decision points, "
        "data transformations, and processes that define how the program achieves its business objective. You translate "
        "technical flow into logical steps."
    ),
    verbose=True,
    allow_delegation=False,
    llm=llm
)

# 3. Diagram Blocks Generator Agent
# This agent translates the logical steps identified by the reviewer into
# abstract diagram blocks. It decides what should be a process, a decision,
# an input/output, etc., without worrying about the final syntax.
diagram_blocks_generator = Agent(
    role='Workflow Diagram Architect',
    goal='Translate the identified business logic into a list of abstract diagram components (e.g., Start, End, Process, Decision, IO).',
    backstory=(
        "You are a systems architect who thinks visually. You can take a description of a process and immediately see it as a "
        "flowchart. You are an expert in UML and other diagramming methodologies, but your current focus is on defining the "
        "fundamental building blocks of the diagram—the nodes and their types—before they are rendered into a specific format."
    ),
    verbose=True,
    allow_delegation=False,
    llm=llm
)

# 4. Diagram Generation Agent
# This agent takes the abstract blocks and generates the final diagram code
# in a specific format, in this case, MermaidJS.
# mermaid_docs_tool = ScrapeWebsiteTool(website_url='https://mermaid.js.org/intro/')

# Define the Mermaid Code Generator agent
diagram_generator = Agent(
    role='Mermaid Code Generator',
    goal='Generate syntactically correct Mermaid code for various diagrams from a natural language description.',
    backstory=(
        "You are an expert in creating diagrams-as-code using Mermaid. You have a deep understanding of Mermaid's syntax "
        "You know the latest syntax of MermaidJS and its common syntax errors and you have always have been known to give the perfect Mermaid code without any errors especially you should not use parantheses inside the each code block of MemaidJS code. "
        "You also do not make the error of using quote inside the code block of MermaidJS code. "
        "for generating a wide variety of diagrams, including flowcharts, sequence diagrams, Gantt charts, and more. "
        "You can translate natural language descriptions or structured data into clean, readable, and accurate Mermaid code. "
        "You often consult the official Mermaid documentation to ensure you are using the latest and most effective features."
    ),
    # The tool that allows the agent to scrape/read websites
    # tools=[mermaid_docs_tool],
    verbose=True,
    allow_delegation=False,
    llm=llm # Pass your configured LLM instance here
)


# 5. Manager Agent (Quality Control)
# This agent acts as the final quality gate. It reviews the generated diagram
# against the initial code to ensure it's a faithful representation—not too
# detailed (e.g., diagramming every single line) and not too high-level
# (e.g., missing critical logic branches).
manager = Agent(
    role='Quality Assurance Manager',
    goal='Review the final MermaidJS diagram to ensure it accurately represents the original ABAP code\'s logic without being overly complex or too simplistic. Provide the final, approved diagram code.',
    backstory=(
        "With a dual background in software development and project management, you are the ultimate quality gatekeeper. "
        "You ensure that the final product meets the initial requirements. Your job is to look at the generated diagram and the "
        "original code and ask: 'Does this diagram help a developer or analyst understand the code's workflow?' You have the final "
        "say on whether the diagram is approved."
        "You check the length and complexity of the diagram as it should neither be too small nor too large or complex but it does not mean that you make it incomplete. The graph should be complete and should not miss any important logic."
    ),
    verbose=True,
    allow_delegation=True,
    llm=llm
)

## File Handling

In [ ]:
%pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.8 MB/s eta 0:00:00


In [ ]:
# 1. Read code from the 'code_files' folder
import PyPDF2 # Import PyPDF2 here
code_folder_path = 'code_files'
if not os.path.exists(code_folder_path):
    os.makedirs(code_folder_path)
    # Create a dummy file for demonstration if the folder is empty
    with open(os.path.join(code_folder_path, 'sample_code.txt'), 'w') as f:
        f.write("REPORT Z_SAMPLE_REPORT.") # Add some dummy code

abap_code_example = ""
for filename in os.listdir(code_folder_path):
    if filename.endswith(".txt"):
        with open(os.path.join(code_folder_path, filename), 'r') as f:
            abap_code_example += f.read() + "\n\n"

print(abap_code_example)

REPORT Z_SAMPLE_REPORT.




/usr/local/lib/python3.11/dist-packages/PyPDF2/__init__.py:21: DeprecationWarning: PyPDF2 is deprecated. Please move to the pypdf library instead.
  warnings.warn(


In [44]:
# --- Task Definitions ---

# Example ABAP code to be processed.
# This code contains a simple report with a start-of-selection event,
# a loop, a conditional statement, and a form call.

# Task for the Interpreter
interpretation_task = Task(
    description=f"""
    Analyze the following ABAP code. Identify the main program events (like START-OF-SELECTION),
    loops (DO/WHILE), conditional logic (IF/ELSE), and subroutine calls (PERFORM).
    Create a clear, summarized list of these structural components and their sequence.

    ABAP Code:
    ---
    {abap_code_example}
    ---
    """,
    expected_output="A structured text summary outlining the main execution flow, loops, conditions, and subroutine calls in the ABAP code.",
    agent=abap_interpreter
)

# Task for the Logic Reviewer
logic_review_task = Task(
    description="""
    Based on the structured summary of the ABAP code, map out the business logic.
    Focus on the sequence of operations and the decisions being made.
    For example, describe the flow like: '1. Start. 2. Loop 5 times. 3. Inside loop, check if counter is even. 4. If even, do X. 5. If odd, do Y. 6. End loop. 7. End.'
    """,
    expected_output="A step-by-step description of the program's logic flow, highlighting decision points and repeated processes.",
    agent=logic_reviewer,
    context=[interpretation_task] # This task depends on the output of the interpretation_task
)

# Task for the Diagram Blocks Generator
block_generation_task = Task(
    description="""
    Convert the step-by-step logic flow into a list of abstract diagram blocks.
    Use standard flowchart notation:
    - 'Start' for the beginning
    - 'End' for the end
    - 'Process' for an action (e.g., 'Initialize variables', 'Write message')
    - 'Decision' for a condition (e.g., 'Is counter even?')
    - 'IO' for any input/output operations.
    - 'Loop' for start/end of loops.

    Define the connections between these blocks.
    """,
    expected_output="A list of objects or a structured text defining each diagram block (e.g., {id: 'A', type: 'Start', label: 'Start Program'}), and the connections between them (e.g., 'A --> B').",
    agent=diagram_blocks_generator,
    context=[logic_review_task]
)

# Task for the Diagram Generator
diagram_generation_task = Task(
    description="""
   Generate the Mermaid code for a flowchart based on a natural language description.
  The diagram should represent a simple user login process, with correct syntax for nodes and connections.
    The flowchart should start with a user entering their credentials.
    """,
    expected_output="A single code block containing the complete, syntactically correct  Mermaid code for the UML activity diagram.",
    agent=diagram_generator,
    context=[block_generation_task]
)

# Task for the Manager
manager_review_task = Task(
    description=f"""
    Review the generated MermaidJS code. Compare it against the original ABAP code's logic to ensure accuracy and appropriate detail.
    Do check the length and complexity of the diagram as it should neither be too small nor too large or complex but it does not mean that you make it incomplete. The graph should be complete and should not miss any important logic.
    The diagram should clearly show the main loop, the conditional branch (even/odd check), and the subroutine call.
    It should not be cluttered with unnecessary details like variable declarations.
    If the diagram is accurate, provide the final MermaidJS code block as your final answer. If not, provide feedback (though for this run, assume it's correct).

    Original ABAP Code for reference:
    ---
    {abap_code_example}
    ---
    """,
    expected_output="The final, approved MermaidJS code block, ready for rendering.",
    agent=manager,
    context=[diagram_generation_task]
)

In [45]:
# --- Crew Definition ---

# Assemble the agents and tasks into a crew
abap_to_diagram_crew = Crew(
    agents=[abap_interpreter, logic_reviewer, diagram_blocks_generator, diagram_generator, manager],
    tasks=[interpretation_task, logic_review_task, block_generation_task, diagram_generation_task, manager_review_task],
    process=Process.sequential,  # The tasks will be executed one after another
    verbose= True  # Provides detailed logs of the process
)


# --- Execution ---

if __name__ == "__main__":
    print("🚀 Starting the ABAP to Diagram Crew with Gemini Pro...")
    print("======================================================")
    mermaid_code = abap_to_diagram_crew.kickoff(
    inputs={
        'abap_code': abap_code_example
    }
)

    print("\n========================================")
    print("✅ Crew execution finished!")
    print("\nFinal Result (MermaidJS Code):")
    print("----------------------------------------")
    # The final result is the output of the last task
    print(mermaid_code)
    print("----------------------------------------")

🚀 Starting the ABAP to Diagram Crew with Gemini Pro...


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 854897ac-c852-4843-a3e1-6948f993ff44                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: ABAP Code Interpreter                                                                                   │
│                                                                                                                 │
│  Task:                                                                                                          │
│      Analyze the following ABAP code. Identify the main program events (like START-OF-SELECTION),               │
│      loops (DO/WHILE), conditional logic (IF/ELSE), and subroutine calls (PERFORM).                             │
│      Create a clear, summarized list of these structural components and their sequence.                         │
│                                                                                                                 │
│      ABAP Code:                                                                                                 │
│      ---                                                                                                        │
│      *&---------------------------------------------------------------------*                                   │
│  *&  Include           ZGNMTD0475_CONSIGN_PROCESS_TOP                                                           │
│  *&---------------------------------------------------------------------*                                       │
│  ************************************************************************                                       │
│  * McCain                                                                                                       │
│  * Global/Local  : Local                                                                                        │
│  * List Site(s)  : SAF                                                                                          │
│  * FS Document ID: MTD_SAF_12_E01                                                                               │
│  * TS Document ID: MTD_SAF_12_TS_E01                                                                            │
│  * TS Title      : Consignment Process Enhancement                                                              │
│  * CTS No        : ED2K913904                                                                                   │
│  * Object Description: Program to be attached to output type ZCIT                                               │
│  *                     (Consignment in Transit) which will be triggered                                         │
│  *                     on PGI of the delivery. TOP include for global                                           │
│  *                      declarations.                                                                           │
│  ************************************************************************                                       │
│  * Revision Log:                                                                                                │
│  * Init.  Author/Reviser  Date     Description of Change CTS# of Changes                                        │
│  * 1   Sidharth Chauhan  29/10/2013 Initial version      ED2K913904                                             │
│  ************************************************************************                                       │
│                                                                                                                 │
│  CONSTANTS: c_dummy_batch TYPE charg_d    VALUE 'CONSGNMENT',                                                   │
│             c_exp_date    TYPE sy-datum   VALUE '99991231',                                                     │
│             c_msgty_e     TYPE msgty      VALUE 'E',   

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: ABAP Code Interpreter                                                                                   │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ### **ABAP Code Analysis**                                                                                     │
│                                                                                                                 │
│  This report provides a structured analysis of the provided ABAP include program `ZGNMTDSR162489_QCD`. The      │
│  code's primary purpose is to dynamically calculate the inspection end date and time for a Quality Management   │
│  (QM) inspection lot (`QALS`) during a goods movement process (like in transaction MIGO). It does this by       │
│  considering the material's QM setup, the movement type, and data from a custom table related to Post-Goods     │
│  Issue (PGI).                                                                                                   │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ### **1. Data Declarations**                                                                                   │
│                                                                                                                 │
│  The program begins by defining local data types and declaring internal tables, work areas, and variables.      │
│                                                                                                                 │
│  *   **Custom Types:**                                                                                          │
│      *   `ty_qmat`: Structure for QM Inspection Setup data (QMAT).                                              │
│      *   `ty_qals`: Structure for Inspection Lot Record data (QALS).                                            │
│  *   **Internal Tables:**                                                                                       │
│      *   `li_qmat`: To hold QM material data from table `QMAT`.                                                 │
│      *   `li_cldpgi`: To hold data from the custom table `ZGMTD_CLD_PGI`.                                       │
│      *   `li_qals`: To hold existing inspection lot data from table `QALS`.                                     │
│      *   `li_callstack`: To hold the program execution call stack.                                              │
│  *   **Work Areas & Variables:**                                                                                │
│      *   Various work areas (`lw_qals`, `lwa_cldpgi`, `lw_qmat`, etc.) for processing single records.           │
│      *   Variables for calculations (`l_adddys`, `l_seconds`, `lv_timestamp`, etc.).                            │
│      *   Variables (`l_text*`) to hold program/variable names for dynamic memory access.                        │
│  *   **Field Symbols:**                                                                                         │
│      *   `<ls_gohead>`, `<ls_goitem>`, `<ls_imseg>`, `<ls_mseg>`: Used to dynamically access data from the      │
│  memory of the calling goods movement programs (e.g., `SAPLMIGO`).                                              │
│                                                        

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 62d325f8-0f3b-4d27-9183-96ca2dfba86b                                                                     │
│  Agent: ABAP Code Interpreter                                                                                   │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

HTTPSConnectionPool(host='telemetry.crewai.com', port=4319): Max retries exceeded with url: /v1/traces (Caused by 
SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get 
local issuer certificate (_ssl.c:1028)')))

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Quality Assurance Manager                                                                               │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```mermaid                                                                                                     │
│  flowchart TD                                                                                                   │
│      %% Node Definitions                                                                                        │
│      A("Start")                                                                                                 │
│      B{"Current T-code is 'MB1B'?"}                                                                             │
│      C["Inspect call stack for 'SAPLMBWL' and 'IMSEG'"]                                                         │
│      D{"Call stack analysis successful?"}                                                                       │
│      E["Set context flag lv_flag = 'X'"]                                                                        │
│      F{"Is context flag lv_flag = 'X'?"}                                                                        │
│      G[/"Access goods movement item data"/]                                                                     │
│      H{"Data access successful?"}                                                                               │
│      I{"User action is 'Check' (OK_CHECK)?"}                                                                    │
│      J[/"Query QMAT for inspection type '08' or 'Z01'"/]                                                        │
│      K{"Material configured for '08' or 'Z01'?"}                                                                │
│      L{"Insp. Type is ('08' or 'Z01') AND Mvmt Type is ('101', '322', '349', 'Z49')?"}                          │
│      M{"Movement Type is '101'?"}                                                                               │
│      N[/"PATH A: Query custom table ZGMTD_CLD_PGI"/]                                                            │
│      O[/"PATH B: Search QALS for previous inspection lot of type '04'"/]                                        │
│      P{"Type '04' lot found?"}                                                                                  │
│      Q[/"Search QALS for previous lot of type '08' or 'Z01'"/]                                                  │
│      R{"PGI records found from ZGMTD_CLD_PGI?"}                                                                 │
│      S["Loop Start: For each PGI record"]                                                                       │
│      T["Convert timestamp (hours) to seconds"]                                                                  │
│      U["Convert lot start date/time to combined timestamp"]                                                     │
│      V["Add seconds to start timestamp for new end timestamp"]                                                  │
│      W["Convert new end timestamp back to date and time"]                                                       │
│      X["Overwrite inspection lot end date/time"]                                                                │
│      Y["Loop End"]                                                                                              │
│      Z{"Previous inspection lots found in QALS?"}                                                               │
│      AA["Sort found lots by end date (descending)"]    

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 8640a962-cf02-4d57-9c69-f09d8df9ff1f                                                                     │
│  Agent: Quality Assurance Manager                                                                               │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 854897ac-c852-4843-a3e1-6948f993ff44                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ```mermaid                                                                                       │
│  flowchart TD                                                                                                   │
│      %% Node Definitions                                                                                        │
│      A("Start")                                                                                                 │
│      B{"Current T-code is 'MB1B'?"}                                                                             │
│      C["Inspect call stack for 'SAPLMBWL' and 'IMSEG'"]                                                         │
│      D{"Call stack analysis successful?"}                                                                       │
│      E["Set context flag lv_flag = 'X'"]                                                                        │
│      F{"Is context flag lv_flag = 'X'?"}                                                                        │
│      G[/"Access goods movement item data"/]                                                                     │
│      H{"Data access successful?"}                                                                               │
│      I{"User action is 'Check' (OK_CHECK)?"}                                                                    │
│      J[/"Query QMAT for inspection type '08' or 'Z01'"/]                                                        │
│      K{"Material configured for '08' or 'Z01'?"}                                                                │
│      L{"Insp. Type is ('08' or 'Z01') AND Mvmt Type is ('101', '322', '349', 'Z49')?"}                          │
│      M{"Movement Type is '101'?"}                                                                               │
│      N[/"PATH A: Query custom table ZGMTD_CLD_PGI"/]                                                            │
│      O[/"PATH B: Search QALS for previous inspection lot of type '04'"/]                                        │
│      P{"Type '04' lot found?"}                                                                                  │
│      Q[/"Search QALS for previous lot of type '08' or 'Z01'"/]                                                  │
│      R{"PGI records found from ZGMTD_CLD_PGI?"}                                                                 │
│      S["Loop Start: For each PGI record"]                                                                       │
│      T["Convert timestamp (hours) to seconds"]                                                                  │
│      U["Convert lot start date/time to combined timestamp"]                                                     │
│      V["Add seconds to start timestamp for new end timestamp"]                                                  │
│      W["Convert new end timestamp back to date and time"]                                                       │
│      X["Overwrite inspection lot end date/time"]                                                                │
│      Y["Loop End"]                                                                                              │
│      Z{"Previous inspection lots found in QALS?"}     


✅ Crew execution finished!

Final Result (MermaidJS Code):
----------------------------------------
```mermaid
flowchart TD
    %% Node Definitions
    A("Start")
    B{"Current T-code is 'MB1B'?"}
    C["Inspect call stack for 'SAPLMBWL' and 'IMSEG'"]
    D{"Call stack analysis successful?"}
    E["Set context flag lv_flag = 'X'"]
    F{"Is context flag lv_flag = 'X'?"}
    G[/"Access goods movement item data"/]
    H{"Data access successful?"}
    I{"User action is 'Check' (OK_CHECK)?"}
    J[/"Query QMAT for inspection type '08' or 'Z01'"/]
    K{"Material configured for '08' or 'Z01'?"}
    L{"Insp. Type is ('08' or 'Z01') AND Mvmt Type is ('101', '322', '349', 'Z49')?"}
    M{"Movement Type is '101'?"}
    N[/"PATH A: Query custom table ZGMTD_CLD_PGI"/]
    O[/"PATH B: Search QALS for previous inspection lot of type '04'"/]
    P{"Type '04' lot found?"}
    Q[/"Search QALS for previous lot of type '08' or 'Z01'"/]
    R{"PGI records found from ZGMTD_CLD_PGI?"}
    S["Loop Start: 

In [46]:
# Dummy placeholder for generated mermaid code
# Replace this with your actual mermaid code logic
def remove_first_and_last_line(text):
    lines = text.splitlines()
    if len(lines) <= 2:
        return ''  # Not enough lines to keep anything
    return '\n'.join(lines[1:-1])

final_mermaid = remove_first_and_last_line(str(mermaid_code))

# Wrap the Mermaid code in a minimal HTML with CDN
html_content = f"""
<!DOCTYPE html>
<html>
<head>
  <meta charset="UTF-8">
  <title>Mermaid Diagram</title>
  <script type="module">
    import mermaid from 'https://cdn.jsdelivr.net/npm/mermaid@10/dist/mermaid.esm.min.mjs';
    mermaid.initialize({{ startOnLoad: true }});
  </script>
</head>
<body>
  <div class="mermaid">
    {final_mermaid}
  </div>
</body>
</html>
"""

# Save the HTML file in current directory
if output_html_filename:
    with open(output_html_filename, 'w', encoding='utf-8') as f:
        f.write(html_content)
    print(f"HTML file saved as: {output_html_filename}")
else:
    print("No output filename found. Make sure Block 1 ran first.")


NameError: name 'mermaid_code' is not defined

In [47]:
import os
import requests

HCTI_API_USER_ID = os.getenv("HCTI_USER_ID") 
HCTI_API_KEY = os.getenv("HCTI_API_KEY")


# Generate the base filename without extension
image_filename = os.path.splitext(output_html_filename)[0] + ".png"

# Send request to HCTI (assuming credentials and data already set)
response = requests.post(
    url="https://hcti.io/v1/image",
    data={
        'html': html_content  # Use the HTML content generated earlier
    },
    auth=(HCTI_API_USER_ID, HCTI_API_KEY),
    verify=False  # Disable SSL verification for this request (not recommended for production use)
)

if response.status_code == 200:
    image_url = response.json()['url']
    image_data = requests.get(image_url, verify=False).content

    with open(image_filename, "wb") as f:
        f.write(image_data)

    print(f"✅ Image saved as: {image_filename}")
else:
    print("❌ Error:", response.status_code, response.text)


✅ Image saved as: 2. ZGNMTDSR162489_02.png


In [40]:
import os
import requests
from tkinter import Tk
from tkinter.filedialog import askopenfilenames

# Set HCTI credentials from environment variables
HCTI_API_USER_ID = os.getenv("HCTI_USER_ID")
HCTI_API_KEY = os.getenv("HCTI_API_KEY")

# Step 1: Open file dialog to choose multiple HTML files
Tk().withdraw()  # Hide the root window
html_paths = askopenfilenames(filetypes=[("HTML files", "*.html")])

if not html_paths:
    print("❌ No files selected.")
    exit()

# Step 2: Loop through selected files
for html_path in html_paths:
    with open(html_path, 'r', encoding='utf-8') as f:
        html_content = f.read()

    image_filename = os.path.splitext(html_path)[0] + ".png"

    # Step 3: Send to HCTI
    response = requests.post(
        url="https://hcti.io/v1/image",
        data={'html': html_content},
        auth=(HCTI_API_USER_ID, HCTI_API_KEY),
        verify=False
    )

    if response.status_code == 200:
        image_url = response.json()['url']
        image_data = requests.get(image_url, verify=False).content

        with open(image_filename, "wb") as f:
            f.write(image_data)

        print(f"✅ Saved: {image_filename}")
    else:
        print(f"❌ Failed for: {html_path} | Error:", response.status_code, response.text)


✅ Saved: C:/Users/akikumar/FSTS/DocReversalEngine/Workflow/RTR Workflow/1.ZGRTR_FI_HRC_CREDIT_UPD.png
✅ Saved: C:/Users/akikumar/FSTS/DocReversalEngine/Workflow/RTR Workflow/2 zgrtr_fi_hrc_credit_top.png
✅ Saved: C:/Users/akikumar/FSTS/DocReversalEngine/Workflow/RTR Workflow/3. zgrtr_fi_hrc_credit_sel.png
✅ Saved: C:/Users/akikumar/FSTS/DocReversalEngine/Workflow/RTR Workflow/3zgrtr_fi_hrc_credit_f01.png
